In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
import keras

DATAPATH = "../data/"
# DATAPATH = "F:/data/"
encoding_dim = 20 # 压缩后的维度

Using TensorFlow backend.


In [2]:
country_feature = np.load(DATAPATH + "features/country_feature.npy")
genre_feature = np.load(DATAPATH + "features/genre_feature.npy")
audio_feature = np.load(DATAPATH + "features/audio_feature.npy")
video_feature = np.load(DATAPATH + "features/video_feature.npy")

# 图像特征归一化
max = video_feature.max()
min = video_feature.min()
video_feature = (video_feature-min)/(max-min)

In [3]:
# 将nan填充为0
audio_feature[np.isnan(audio_feature)] = 0.

In [4]:
country_input = Input(shape=(country_feature.shape[1],))
genre_input = Input(shape=(genre_feature.shape[1],))
audio_input = Input(shape=(audio_feature.shape[1],))
video_input = Input(shape=(video_feature.shape[1],))

In [5]:
input_layer = keras.layers.concatenate([country_input, genre_input, audio_input, video_input])

# encoder layers
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(10, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded) # 不设置激励函数则正负都有

# decoder layers
decoded = Dense(10, activation='relu')(encoder_output)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='relu')(decoded)

In [6]:
# output layers
'''
softmax [0,1]，每一行的和为1
sigmoid [0,1]，每一行的元素大小相互独立
tanh [-1,1]，每一行的元素大小互相独立
'''
country_output = Dense(country_feature.shape[1], activation='softmax')(decoded)
genre_output = Dense(genre_feature.shape[1], activation='sigmoid')(decoded)
audio_output = Dense(audio_feature.shape[1], activation='tanh')(decoded)
video_output = Dense(video_feature.shape[1], activation='sigmoid')(decoded)

In [7]:
# construct the autoencoder model
autoencoder = Model(input=[country_input, genre_input, audio_input, video_input],
                    output=[country_output, genre_output, audio_output, video_output])

# construct the encoder model for plotting
encoder = Model(input=[country_input, genre_input, audio_input, video_input], output=encoder_output)

# compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# training
autoencoder.fit([country_feature, genre_feature, audio_feature, video_feature],
                [country_feature, genre_feature, audio_feature, video_feature],
                epochs=5, batch_size=256, shuffle=True, verbose=1)

c:\users\wang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  after removing the cwd from sys.path.
c:\users\wang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  import sys


Epoch 1/5
2/2 [==============================] - 1s 522ms/step - loss: 0.4718 - dense_9_loss: 0.0263 - dense_10_loss: 0.2496 - dense_11_loss: 0.0129 - dense_12_loss: 0.1830
Epoch 2/5
2/2 [==============================] - 0s 64ms/step - loss: 0.4621 - dense_9_loss: 0.0263 - dense_10_loss: 0.2407 - dense_11_loss: 0.0128 - dense_12_loss: 0.1823
Epoch 3/5
2/2 [==============================] - 0s 232ms/step - loss: 0.4386 - dense_9_loss: 0.0262 - dense_10_loss: 0.2191 - dense_11_loss: 0.0138 - dense_12_loss: 0.1795
Epoch 4/5
2/2 [==============================] - 0s 47ms/step - loss: 0.3998 - dense_9_loss: 0.0260 - dense_10_loss: 0.1845 - dense_11_loss: 0.0168 - dense_12_loss: 0.1726
Epoch 5/5
2/2 [==============================] - 0s 78ms/step - loss: 0.3468 - dense_9_loss: 0.0256 - dense_10_loss: 0.1407 - dense_11_loss: 0.0207 - dense_12_loss: 0.1597


In [8]:
test = autoencoder.predict([country_feature, genre_feature, audio_feature, video_feature])
test

[array([[0.02535267, 0.03503346, 0.03677804, 0.02840181, 0.02904468,
         0.0256127 , 0.023434  , 0.03180755, 0.02061424, 0.02260449,
         0.02641176, 0.02064174, 0.02600011, 0.03688079, 0.03334985,
         0.02107145, 0.0247786 , 0.02006006, 0.01886956, 0.02762963,
         0.02428477, 0.02870151, 0.02199638, 0.02866681, 0.02533287,
         0.02216058, 0.01841056, 0.02294583, 0.02515926, 0.02678742,
         0.02483467, 0.02762625, 0.03800858, 0.02249646, 0.05441941,
         0.01971539, 0.03407622],
        [0.02407982, 0.03893843, 0.04156536, 0.02846734, 0.02940315,
         0.02446909, 0.02148611, 0.03349532, 0.01787039, 0.02048781,
         0.02564028, 0.01789319, 0.02503012, 0.04197232, 0.03599644,
         0.01839658, 0.02329893, 0.01722548, 0.01567552, 0.02728033,
         0.02253904, 0.02893565, 0.01948475, 0.02881265, 0.02403709,
         0.01981138, 0.01514138, 0.02067968, 0.0237397 , 0.02631097,
         0.02335338, 0.0274784 , 0.04350316, 0.02028986, 0.07348394,


In [9]:
test_encoder = encoder.predict([country_feature, genre_feature, audio_feature, video_feature])
print(test_encoder)
np.save(DATAPATH + "features/low_feature.npy", test_encoder)
print("save success!")

[[-0.36497107  3.2004347   1.279102    0.24324037 -1.0158732  -0.12871471
   0.92512244  2.480282    1.7005184   0.26384318 -1.3649292   1.475345
  -0.1133886  -2.3323855  -0.79558575 -3.0912225  -2.0175076   1.2519102
   2.3600605   1.3670928 ]
 [-0.4690487   4.7259216   1.8322754   0.33657908 -1.4418467  -0.20096529
   1.4103242   3.6599803   2.4337568   0.4776803  -1.9812964   2.2830224
  -0.28148425 -3.3615735  -1.2084526  -4.598768   -2.9371529   1.8675063
   3.5844016   2.1629822 ]]
save success!
